## Exercise M3.01

The goal is to write an exhaustive search to find the best parameters combination maximizing the model generalization performance.

Here we use a small subset of the Adult Census dataset to make the code faster to execute. Once your code works on the small subset, try to change train_size to a larger value (e.g. 0.8 for 80% instead of 20%).

In [6]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

In [4]:
from sklearn.model_selection import train_test_split

adult_census = pd.read_csv("/content/drive/MyDrive/DataSets/Adult_Census/adult-census(1).csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.8, random_state=42
)

In [5]:
categorical_preprocessor = OrdinalEncoder(
    handle_unknown="use_encoded_value", unknown_value=-1
)
preprocessor = ColumnTransformer(
    [
        (
            "cat_preprocessor",
            categorical_preprocessor,
            selector(dtype_include=object),
        )
    ],
    remainder="passthrough",
)

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("classifier", HistGradientBoostingClassifier(random_state=42)),
    ]
)

Use the previously defined model (called model) and using two nested for loops, make a search of the best combinations of the learning_rate and max_leaf_nodes parameters. In this regard, you have to train and test the model by setting the parameters. The evaluation of the model should be performed using cross_val_score on the training set. Use the following parameters search:

    learning_rate for the values 0.01, 0.1, 1 and 10. This parameter controls the ability of a new tree to correct the error of the previous sequence of trees
    max_leaf_nodes for the values 3, 10, 30. This parameter controls the depth of each tree.



In [7]:
learning_rates = [0.01, 0.1, 1, 10]
max_leaf_nodes_values = [3, 10, 30]

best_score = 0
best_params = {}

for learning_rate in learning_rates:
    for max_leaf_nodes in max_leaf_nodes_values:

        model.set_params(classifier__learning_rate=learning_rate,
                         classifier__max_leaf_nodes=max_leaf_nodes)


        scores = cross_val_score(model, data_train, target_train, cv=5)


        mean_score = scores.mean()


        if mean_score > best_score:
            best_score = mean_score
            best_params = {'learning_rate': learning_rate, 'max_leaf_nodes': max_leaf_nodes}

# Print the best score and the corresponding parameters
print(f"Best score: {best_score}")
print(f"Best parameters: {best_params}")

Best score: 0.8714456736526677
Best parameters: {'learning_rate': 0.1, 'max_leaf_nodes': 30}


Now use the test set to score the model using the best parameters that we found using cross-validation. You will have to refit the model over the full training set.

In [8]:
model.set_params(classifier__learning_rate=best_params['learning_rate'],
                 classifier__max_leaf_nodes=best_params['max_leaf_nodes'])

model.fit(data_train, target_train)

test_score = model.score(data_test, target_test)

print(f"Test score: {test_score}")


Test score: 0.8789026512437301
